In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY")

In [6]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o",temperature=0.5)

C:\Users\ZhangBeixiang\AppData\Local\Temp\ipykernel_22156\1729027061.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o",temperature=0.5)


In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple-choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple-choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple-choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz for {number} multiple-choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be comfirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Remember to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\ZhangBeixiang\AppData\Local\Temp\ipykernel_22156\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [12]:
TEMPLATE2="""
You are an expert English grammarian and writer. Given a Multiple-Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
If the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template = TEMPLATE2
)

In [14]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain], 
                                        input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz","review"], verbose=True)

In [16]:
file_path = r"E:\tmcqgen\data.txt"

In [17]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [19]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple-choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple-choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple-choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [32]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="simple"

In [33]:
# How to set up Token Usage Tracking in Langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [34]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 2359
Prompt Tokens: 1416
Completion Tokens: 943
Total Cost: 0.021225


In [35]:
# Let's try to get the response
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [36]:
quiz = response.get("quiz")

In [37]:
print(quiz)

{
  "1": {
    "mcq": "Who coined the term 'machine learning' in 1959?",
    "options": {
      "a": "Donald Hebb",
      "b": "Alan Turing",
      "c": "Arthur Samuel",
      "d": "Walter Pitts"
    },
    "correct": "Arthur Samuel"
  },
  "2": {
    "mcq": "What was the name of the early experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
    "options": {
      "a": "Cybertron",
      "b": "NeuroNet",
      "c": "AlphaGo",
      "d": "DeepBlue"
    },
    "correct": "Cybertron"
  },
  "3": {
    "mcq": "Which book by Donald Hebb introduced a theoretical neural structure formed by interactions among nerve cells?",
    "options": {
      "a": "The Organization of Behavior",
      "b": "Learning Machines",
      "c": "Artificial Intelligence",
      "d": "Neural Networks"
    },
    "correct": "The Organization of Behavior"
  },
  "4": {
    "mcq": "Who provided a widely quoted formal definition of the algorithms studied in the machine learning field?",
 

In [38]:
review = response.get("review")

In [39]:
print(review)

### Complexity Analysis
The quiz questions are generally appropriate for students studying machine learning, as they cover historical figures and foundational concepts. However, some questions may require specific prior knowledge that not all students might possess.

### Updated Quiz Questions
To better match the cognitive and analytical abilities of machine learning students, slight adjustments in tone and phrasing can be made:

```json
{
  "1": {
    "mcq": "Who introduced the term 'machine learning' in 1959?",
    "options": {
      "a": "Donald Hebb",
      "b": "Alan Turing",
      "c": "Arthur Samuel",
      "d": "Walter Pitts"
    },
    "correct": "Arthur Samuel"
  },
  "2": {
    "mcq": "What was the name of the early experimental 'learning machine' developed by Raytheon Company in the 1960s?",
    "options": {
      "a": "Cybertron",
      "b": "NeuroNet",
      "c": "AlphaGo",
      "d": "DeepBlue"
    },
    "correct": "Cybertron"
  },
  "3": {
    "mcq": "Which book by Don

In [40]:
# Let's convert the format of the result
# quiz is str now
type(quiz)

str

In [42]:
quiz=json.loads(quiz)

In [43]:
# quiz is dict now
type(quiz)

dict

In [44]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [45]:
print(quiz_table_data)

[{'MCQ': "Who coined the term 'machine learning' in 1959?", 'Choices': 'a: Donald Hebb | b: Alan Turing | c: Arthur Samuel | d: Walter Pitts', 'Correct': 'Arthur Samuel'}, {'MCQ': "What was the name of the early experimental 'learning machine' developed by Raytheon Company in the early 1960s?", 'Choices': 'a: Cybertron | b: NeuroNet | c: AlphaGo | d: DeepBlue', 'Correct': 'Cybertron'}, {'MCQ': 'Which book by Donald Hebb introduced a theoretical neural structure formed by interactions among nerve cells?', 'Choices': 'a: The Organization of Behavior | b: Learning Machines | c: Artificial Intelligence | d: Neural Networks', 'Correct': 'The Organization of Behavior'}, {'MCQ': 'Who provided a widely quoted formal definition of the algorithms studied in the machine learning field?', 'Choices': 'a: Arthur Samuel | b: Tom M. Mitchell | c: Warren McCulloch | d: Walter Pitts', 'Correct': 'Tom M. Mitchell'}, {'MCQ': 'What are the two main objectives of modern-day machine learning?', 'Choices': 'a

In [47]:
quiz_formatted = pd.DataFrame(quiz_table_data)

In [48]:
quiz_formatted.to_csv("machine_learning_quiz.csv", index=False)